Bộ dữ liệu chạy thử nghiệm là bộ dl lấy trong bài báo Improving protein succinylation sites prediction using embeddings from protein language model
link: https://www.nature.com/articles/s41598-022-21366-2

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle as cPickle

from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc

import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Bidirectional, LSTM, SimpleRNN, Input, GRU
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Embedding, Dropout, Activation, MaxPooling1D,Conv1D
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.layers import Dense, BatchNormalization, LeakyReLU
from tensorflow.keras.initializers import glorot_uniform
from sklearn.metrics import confusion_matrix, matthews_corrcoef, accuracy_score,roc_auc_score
import sklearn.metrics as metrics
from keras import models

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path_train= "/content/drive/MyDrive/CBiLSuccSite/Data/Train/"
path_test = "/content/drive/MyDrive/CBiLSuccSite/Data/Test/"

path_result = "/content/drive/MyDrive/CBiLSuccSite/Result/"
path_model ="/content/drive/MyDrive/CBiLSuccSite/Model/"

filename_train = "train_data.csv"


In [ ]:
file_train = filename_train
df_train = pd.read_csv(path_train + file_train,  delimiter= ',')


In [ ]:
#def Dictionary for 1-gram
def Dic_1_gram():
    AA_list_sort = ['G','A','V','L','I','M','P','F','W','S','T','N','Q','Y','C','K','R','H','D','E','X']

    AA_dict = {}
    numm = 1
    for i in AA_list_sort:
        AA_dict[i] = numm
        numm += 1
    return AA_dict
# Separation of words in protein sequence
def ProSentence(pro, K):
	sentence = ""
	length = len(pro)
	for i in range(length - K + 1):
		sentence += pro[i: i + K] + " "
    #delete extra space
	sentence = sentence[0 : len(sentence) - 1]
	return sentence

In [ ]:
k =1#1-gram
word_index1 =  Dic_1_gram()
vocab_size = len(word_index1)

In [ ]:
# Preprocessing for train data
texts_train =[] #PTMsequend kmer
for i in df_train['Sequence']:
  temp = ProSentence(i,k)
  texts_train.append(temp)
df_train['k_mer'] =texts_train
train_sequences = []
for each in texts_train:
    each_index_list = []
    each = each.split(' ')
    for i in each:
        each_index_list.append(word_index1[i])
    train_sequences.append(each_index_list)
# Tokenizer train data
data_token = []
for i in df_train['k_mer']:
   data_token.append(i.split())

MAX_SEQUENCE_LENGTH = len(data_token[1])
Xtrain = pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
ytrain = df_train['Label']
Xtrain.shape


(9499, 33)

In [ ]:
lr = 0.0001
num_epochs = 80
TIME_STEPS = MAX_SEQUENCE_LENGTH
INPUT_SIZE = 300

In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers import Conv1D, MaxPool1D, Flatten, Dense,GlobalAveragePooling1D

from keras.layers import concatenate
from tensorflow.keras import Model
from keras.layers import Dense,Embedding,LSTM,Dropout,Bidirectional,GRU, Flatten
def Classifier_CNN_BiLSTM(Xtrain,ytrain, Xtest, ytest):
    model1 = Sequential()
  #input
    model1.add(Embedding(vocab_size+1, 300, input_length=MAX_SEQUENCE_LENGTH,trainable=True))
    model1.add(Conv1D(64, 3, activation="relu", input_shape=(Xtrain.shape[1],1)))
    model1.add(MaxPool1D())
    model1.add(Dropout(0.4))
    model1.add(Conv1D(64,3, activation ="relu"))#,padding='valid',activation='relu',kernel_initializer='random_uniform', bias_initializer='zeros',strides=1))
    model1.add(MaxPool1D())
    model1.add(Bidirectional(GRU(16, dropout=0.4, recurrent_dropout=0,return_sequences=True)))
    model1.add(GlobalAveragePooling1D())
    model1.add(Flatten())


    model2 = Sequential()

    model2.add(Embedding(vocab_size+1, 300, input_length=MAX_SEQUENCE_LENGTH,trainable=True))
    model2.add(Dropout(0.4))
    model2.add(Bidirectional(LSTM(units=32,batch_input_shape=(None,TIME_STEPS, 300),return_sequences=True)))
    model2.add(Flatten())

    merged = concatenate(([model1.output, model2.output]))
    z = Dense(128, activation="relu")(merged)
    z = Dropout(0.2)(z)

    z = Dense(1, activation="sigmoid")(z)

    model = Model(inputs=[model1.input, model2.input], outputs=z)

    model.compile(optimizer=keras.optimizers.Adam(learning_rate = lr),
                loss='binary_crossentropy',
                metrics=['accuracy',"AUC"])


    return model

In [ ]:
model = Classifier_CNN_BiLSTM(Xtrain, ytrain, Xtest,ytest)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 embedding_input (InputLaye  [(None, 33)]                 0         []                            
 r)                                                                                               
                                                                                                  
 embedding (Embedding)       (None, 33, 300)              6600      ['embedding_input[0][0]']     
                                                                                                  
 conv1d (Conv1D)             (None, 31, 64)               57664     ['embedding[0][0]']           
                                                                                                  
 max_pooling1d (MaxPooling1  (None, 15, 64)               0         ['conv1d[0][0]']          

In [ ]:
model =  Classifier_CNN_BiLSTM(Xtrain,ytrain, Xtest,ytest)
model.fit([Xtrain,Xtrain],ytrain, batch_size=32, epochs=num_epochs)
model.save(path_model +"Suci_CNN_BiLSTM.h5")

Epoch 1/80
297/297 [==============================] - 10s 11ms/step - loss: 0.6822 - accuracy: 0.5578 - auc: 0.5905
Epoch 2/80
297/297 [==============================] - 3s 10ms/step - loss: 0.6525 - accuracy: 0.6139 - auc: 0.6614
Epoch 3/80
297/297 [==============================] - 3s 10ms/step - loss: 0.6176 - accuracy: 0.6611 - auc: 0.7202
Epoch 4/80
297/297 [==============================] - 3s 11ms/step - loss: 0.5840 - accuracy: 0.6917 - auc: 0.7593
Epoch 5/80
297/297 [==============================] - 3s 11ms/step - loss: 0.5627 - accuracy: 0.7080 - auc: 0.7813
Epoch 6/80
297/297 [==============================] - 3s 10ms/step - loss: 0.5459 - accuracy: 0.7254 - auc: 0.7974
Epoch 7/80
297/297 [==============================] - 3s 10ms/step - loss: 0.5351 - accuracy: 0.7370 - auc: 0.8074
Epoch 8/80
297/297 [==============================] - 3s 11ms/step - loss: 0.5247 - accuracy: 0.7407 - auc: 0.8160
Epoch 9/80
297/297 [==============================] - 3s 10ms/step - loss: 0.51

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
